<script type="text/javascript" src="http://cdn.mathjax.org/mathjax/latest/MathJax.js?config=default"></script>
# 3 决策树（decision tree）
   决策树是一种基本的分类和回归方法，决策树呈树形结构，在分类问题中，表示基于特征对实例进行分类的过程。决策树学习主要包括3个步骤：特征选择、决策树的生成和决策树的修剪。下图是一个决策树的示意图，图中圆和方框分别表示内部节点和叶节点。
<div align="center">
<img width='50%' algin='middle' src='Image/1.jpg'>
</div>
## 3.1 决策树学习
   决策树学习，假设给定训练数据集
$$D={(x_1,y_1), (x_2, y_2),...,(x_n, y_n)}$$
其中，$x_i=(x_i^{(1)},x_i^{(2)},...,x_i^{(n)})^T为输入实例，n为特征个数，$y_i\in {1,2,...,K}$为类标记，学习的目的是根据给定训练数据集构建一个决策树模型，使它能够对实例进行正确的分类。
## 3.2 特征选择


In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from math import log

In [4]:
def create_data():
    dataset = [['青绿','蜷缩', '浊响', '清晰', '凹陷', '硬滑', '好'],
                       ['乌黑', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', '好'],
                       ['乌黑',' 蜷缩',' 浊响',' 清晰',' 凹陷', '硬滑', '好'],
                       ['青绿',' 蜷缩',' 沉闷',' 清晰',' 凹陷', '硬滑', '好'],
                       ['浅白',' 蜷缩',' 浊响',' 清晰',' 凹陷', '硬滑', '好'],
                       ['青绿',' 稍蜷',' 浊响',' 清晰',' 稍凹', '软粘', '好'],
                       ['乌黑',' 稍蜷',' 浊响',' 稍糊',' 稍凹', '软粘', '好'], 
                       ['乌黑',' 稍蜷',' 浊响',' 清晰',' 稍凹', '硬滑', '好'],
                       ['乌黑',' 稍蜷',' 沉闷',' 稍糊',' 稍凹', '硬滑', '坏'],
                       ['青绿',' 硬挺',' 清脆',' 清晰',' 平坦', '软粘', '坏'],
                       ['浅白',' 硬挺',' 清脆',' 模糊',' 平坦', '硬滑', '坏'], 
                       ['浅白',' 蜷缩',' 浊响',' 模糊',' 平坦', '软粘', '坏'],
                       ['青绿',' 稍蜷',' 浊响',' 稍糊',' 凹陷', '硬滑', '坏'],
                       ['浅白',' 稍蜷',' 沉闷',' 稍糊',' 凹陷', '硬滑', '坏'],
                       ['乌黑',' 稍蜷',' 浊响',' 清晰',' 稍凹', '软粘', '坏'],
                       ['浅白',' 蜷缩',' 浊响',' 模糊',' 平坦', '硬滑', '坏'],
                       ['青绿',' 蜷缩',' 沉闷',' 稍糊',' 稍凹', '硬滑', '坏']
                      ]
  
    labels = ['色泽', '根底', '敲声', '纹理', '脐部', '触感', '好瓜']
    return dataset, labels
                       

In [5]:
train_data, labels = create_data()
train_data

[['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', '好'],
 ['乌黑', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', '好'],
 ['乌黑', ' 蜷缩', ' 浊响', ' 清晰', ' 凹陷', '硬滑', '好'],
 ['青绿', ' 蜷缩', ' 沉闷', ' 清晰', ' 凹陷', '硬滑', '好'],
 ['浅白', ' 蜷缩', ' 浊响', ' 清晰', ' 凹陷', '硬滑', '好'],
 ['青绿', ' 稍蜷', ' 浊响', ' 清晰', ' 稍凹', '软粘', '好'],
 ['乌黑', ' 稍蜷', ' 浊响', ' 稍糊', ' 稍凹', '软粘', '好'],
 ['乌黑', ' 稍蜷', ' 浊响', ' 清晰', ' 稍凹', '硬滑', '好'],
 ['乌黑', ' 稍蜷', ' 沉闷', ' 稍糊', ' 稍凹', '硬滑', '坏'],
 ['青绿', ' 硬挺', ' 清脆', ' 清晰', ' 平坦', '软粘', '坏'],
 ['浅白', ' 硬挺', ' 清脆', ' 模糊', ' 平坦', '硬滑', '坏'],
 ['浅白', ' 蜷缩', ' 浊响', ' 模糊', ' 平坦', '软粘', '坏'],
 ['青绿', ' 稍蜷', ' 浊响', ' 稍糊', ' 凹陷', '硬滑', '坏'],
 ['浅白', ' 稍蜷', ' 沉闷', ' 稍糊', ' 凹陷', '硬滑', '坏'],
 ['乌黑', ' 稍蜷', ' 浊响', ' 清晰', ' 稍凹', '软粘', '坏'],
 ['浅白', ' 蜷缩', ' 浊响', ' 模糊', ' 平坦', '硬滑', '坏'],
 ['青绿', ' 蜷缩', ' 沉闷', ' 稍糊', ' 稍凹', '硬滑', '坏']]

In [6]:
len(train_data)


17

In [7]:
# 计算熵
def ent_calc(dataset):
    length = len(dataset)
    label_count = {}
    for i in range(length):
        label = dataset[i][-1]
        if label not in label_count:
            label_count[label] = 0
        label_count[label] += 1
    ent = -sum([pk/length*np.log2(pk/length) for pk in label_count.values()])
    return ent

# 计算经验熵
def ent_resid(dataset, axis=0):
    length = len(dataset)
    feature_set =  {}
    for i in range(length):
        f = dataset[i][axis]
        if f not in feature_set:
            feature_set[f] = []
        feature_set[f].append(dataset[i])
#     for p in feature_set.values():
#         print(p)
#         print('\n\n')
#         print(ent_calc(p))

    ent = sum([len(p)/length*ent_calc(p)for p in feature_set.values()])
    return ent

def info_gain(ent_calc, ent_resid):
    return ent_calc-ent_resid

def info_gain_train(dataset):
    columns = len(dataset[0])-1
    ent = ent_calc(dataset)
    best_feature = []
    for i in range(columns):
        cond_ent = ent_resid(dataset, axis=i)
        gain = info_gain(ent, cond_ent)
        best_feature.append((labels[i], gain))
    print(best_feature)
    best_  = max(best_feature, key = lambda x: x[-1])
    return best_

In [8]:
ent_calc(train_data)

0.99750254636911528

In [9]:
ent_resid(train_data)

0.88937738110374998

In [10]:
info_gain_train(train_data)

[('色泽', 0.10812516526536531), ('根底', 0.23887919623736464), ('敲声', 0.28192625011143702), ('纹理', 0.42976816669833717), ('脐部', 0.3589876656471539), ('触感', 0.0060464891765655837)]


('纹理', 0.42976816669833717)

In [11]:
class Node:
    def __init__(self, root=True, label=None, feature_name=None, feature=None):
        self.root = root
        self.label = label
        self.feature_name = feature_name 
        self.feature = feature
        self.tree = {}
        self.result = {'label':self.label, 'feature':self.feature, 'tree':self.tree}
    def add_node(self, val, node):
        self.tree[val] = node
    
    def predict(self, features):
        if self.root is True:
            return self.label
        return self.tree[features[self.feature]].predict(features)

In [24]:
class DecisionTree:
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon
        self._tree = {}
    
# 计算熵
    def ent_calc(self,dataset):
        length = len(dataset)
        label_count = {}
        for i in range(length):
            label = dataset[i][-1]
            if label not in label_count:
                label_count[label] = 0
            label_count[label] += 1
        ent = -sum([pk/length*np.log2(pk/length) for pk in label_count.values()])
        return ent

# 计算经验熵
    def ent_resid(self,dataset, axis=0):
        length = len(dataset)
        feature_set =  {}
        for i in range(length):
            f = dataset[i][axis]
            if f not in feature_set:
                feature_set[f] = []
            feature_set[f].append(dataset[i])
        ent = sum([len(p)/length*ent_calc(p)for p in feature_set.values()])
        return ent

# 信息增益
    def info_gain(self, ent_calc, ent_resid):
        return ent_calc-ent_resid

# 
    def info_gain_train(self, dataset):
        columns = len(dataset[0])-1
        ent = ent_calc(dataset)
        best_feature = []
        for i in range(columns):
            cond_ent = ent_resid(dataset, axis=i)
            gain = info_gain(ent, cond_ent)
            best_feature.append((i, gain))
        print(best_feature)
        best_  = max(best_feature, key = lambda x: x[-1])
        return best_
# 
    def train(self, train_data):
        _, y_train, features = train_data.iloc[:, :-1], train_data.iloc[:, -1], train_data.columns[:-1]
        
        if len(y_train.value_counts()) == 1:
            return Node(root=True, label=y_train.iloc[0])
        
        if len(features) == 0:
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])
        # 计算最大信息增益，
        max_feature, max_info_gain = self.info_gain_train(train_data.as_matrix())
        max_feature_name = features[max_feature]
        
        # 4.Ag的信息增益小于阈值
        if max_info_gain < self.epsilon:
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])
        
        # 5. 构建Ag子集
        node_tree = Node(root=False, feature_name=max_feature_name, feature=max_feature)
        
        feature_list = train_data[max_feature_name].value_counts().index
        for f in feature_list:
            sub_train_df = train_data.loc[train_data[max_feature_name] == f].drop([max_feature_name], axis=1)
            
            sub_tree = self.train(sub_train_df)
            node_tree.add_node(f, sub_tree)
        return node_tree
    def fit(self, train_data):
        self._tree = self.train(train_data)
        return self._tree
    
    def predict(self, x_test):
        return self._tree.predict(x_test)
        
        
        
        

In [25]:
datasets, labels = create_data()
datasets = pd.DataFrame(datasets, columns=labels)

In [27]:
dt = DecisionTree()
tree = dt.fit(datasets)

[(0, 0.10812516526536531), (1, 0.23887919623736464), (2, 0.28192625011143702), (3, 0.42976816669833717), (4, 0.3589876656471539), (5, 0.0060464891765655837)]
[(0, 0.0760098536627829), (1, 0.46956521111470695), (2, 0.34745764364708642), (3, 0.46956521111470695), (4, 0.46956521111470695)]
[(0, 0.25162916738782293), (1, 0.0), (2, 0.0), (3, 0.25162916738782293)]
[(0, 0.0), (1, 0.0), (2, 1.0)]
[(0, 0.32192809488736229), (1, 0.072905595320056027), (2, 0.32192809488736229), (3, 0.17095059445466865), (4, 0.72192809488736231)]


In [29]:
print(tree)

In [33]:
# 定义节点类 二叉树
class Node:
    def __init__(self, root=True, label=None, feature_name=None, feature=None):
        self.root = root
        self.label = label
        self.feature_name = feature_name
        self.feature = feature
        self.tree = {}
        self.result = {'label:': self.label, 'feature': self.feature, 'tree': self.tree}

    def __repr__(self):
        return '{}'.format(self.result)

    def add_node(self, val, node):
        self.tree[val] = node

    def predict(self, features):
        if self.root is True:
            return self.label
        return self.tree[features[self.feature]].predict(features)
    
class DTree:
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon
        self._tree = {}

    # 熵
    @staticmethod
    def calc_ent(datasets):
        data_length = len(datasets)
        label_count = {}
        for i in range(data_length):
            label = datasets[i][-1]
            if label not in label_count:
                label_count[label] = 0
            label_count[label] += 1
        ent = -sum([(p/data_length)*log(p/data_length, 2) for p in label_count.values()])
        return ent

    # 经验条件熵
    def cond_ent(self, datasets, axis=0):
        data_length = len(datasets)
        feature_sets = {}
        for i in range(data_length):
            feature = datasets[i][axis]
            if feature not in feature_sets:
                feature_sets[feature] = []
            feature_sets[feature].append(datasets[i])
        cond_ent = sum([(len(p)/data_length)*self.calc_ent(p) for p in feature_sets.values()])
        return cond_ent

    # 信息增益
    @staticmethod
    def info_gain(ent, cond_ent):
        return ent - cond_ent

    def info_gain_train(self, datasets):
        count = len(datasets[0]) - 1
        ent = self.calc_ent(datasets)
        best_feature = []
        for c in range(count):
            c_info_gain = self.info_gain(ent, self.cond_ent(datasets, axis=c))
            best_feature.append((c, c_info_gain))
        # 比较大小
        best_ = max(best_feature, key=lambda x: x[-1])
        return best_

    def train(self, train_data):
        """
        input:数据集D(DataFrame格式)，特征集A，阈值eta
        output:决策树T
        """
        _, y_train, features = train_data.iloc[:, :-1], train_data.iloc[:, -1], train_data.columns[:-1]
        # 1,若D中实例属于同一类Ck，则T为单节点树，并将类Ck作为结点的类标记，返回T
        if len(y_train.value_counts()) == 1:
            return Node(root=True,
                        label=y_train.iloc[0])

        # 2, 若A为空，则T为单节点树，将D中实例树最大的类Ck作为该节点的类标记，返回T
        if len(features) == 0:
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])

        # 3,计算最大信息增益 同5.1,Ag为信息增益最大的特征
        max_feature, max_info_gain = self.info_gain_train(np.array(train_data))
        max_feature_name = features[max_feature]

        # 4,Ag的信息增益小于阈值eta,则置T为单节点树，并将D中是实例数最大的类Ck作为该节点的类标记，返回T
        if max_info_gain < self.epsilon:
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])

        # 5,构建Ag子集
        node_tree = Node(root=False, feature_name=max_feature_name, feature=max_feature)

        feature_list = train_data[max_feature_name].value_counts().index
        for f in feature_list:
            sub_train_df = train_data.loc[train_data[max_feature_name] == f].drop([max_feature_name], axis=1)

            # 6, 递归生成树
            sub_tree = self.train(sub_train_df)
            node_tree.add_node(f, sub_tree)

        # pprint.pprint(node_tree.tree)
        return node_tree

    def fit(self, train_data):
        self._tree = self.train(train_data)
        return self._tree

    def predict(self, X_test):
        return self._tree.predict(X_test)

In [34]:
def create_():
    dataset = [        ['青绿','蜷缩', '浊响', '清晰', '凹陷', '硬滑', '1'],
                       ['乌黑', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', '1'],
                       ['乌黑',' 蜷缩',' 浊响',' 清晰',' 凹陷', '硬滑', '1'],
                       ['青绿',' 蜷缩',' 沉闷',' 清晰',' 凹陷', '硬滑', '1'],
                       ['浅白',' 蜷缩',' 浊响',' 清晰',' 凹陷', '硬滑', '1'],
                       ['青绿',' 稍蜷',' 浊响',' 清晰',' 稍凹', '软粘', '1'],
                       ['乌黑',' 稍蜷',' 浊响',' 稍糊',' 稍凹', '软粘', '1'], 
                       ['乌黑',' 稍蜷',' 浊响',' 清晰',' 稍凹', '硬滑', '1'],
                       ['乌黑',' 稍蜷',' 沉闷',' 稍糊',' 稍凹', '硬滑', '0'],
                       ['青绿',' 硬挺',' 清脆',' 清晰',' 平坦', '软粘', '0'],
                       ['浅白',' 硬挺',' 清脆',' 模糊',' 平坦', '硬滑', '0'], 
                       ['浅白',' 蜷缩',' 浊响',' 模糊',' 平坦', '软粘', '0'],
                       ['青绿',' 稍蜷',' 浊响',' 稍糊',' 凹陷', '硬滑', '0'],
                       ['浅白',' 稍蜷',' 沉闷',' 稍糊',' 凹陷', '硬滑', '0'],
                       ['乌黑',' 稍蜷',' 浊响',' 清晰',' 稍凹', '软粘', '0'],
                       ['浅白',' 蜷缩',' 浊响',' 模糊',' 平坦', '硬滑', '0'],
                       ['青绿',' 蜷缩',' 沉闷',' 稍糊',' 稍凹', '硬滑', '0']
                      ]
  
    labels = ['色泽', '根底', '敲声', '纹理', '脐部', '触感', '好瓜']
    return pd.DataFrame(dataset, columns=labels)
                       

In [35]:

datasets, labels = create_data()
data_df = pd.DataFrame(datasets, columns=labels)
dt = DTree()
tree = dt.fit(data_df)

In [36]:
tree

{'label:': None, 'feature': 3, 'tree': {' 清晰': {'label:': None, 'feature': 1, 'tree': {' 蜷缩': {'label:': '好', 'feature': None, 'tree': {}}, ' 稍蜷': {'label:': None, 'feature': 0, 'tree': {'乌黑': {'label:': None, 'feature': 2, 'tree': {'硬滑': {'label:': '好', 'feature': None, 'tree': {}}, '软粘': {'label:': '坏', 'feature': None, 'tree': {}}}}, '青绿': {'label:': '好', 'feature': None, 'tree': {}}}}, ' 硬挺': {'label:': '坏', 'feature': None, 'tree': {}}}}, ' 稍糊': {'label:': None, 'feature': 4, 'tree': {'硬滑': {'label:': '坏', 'feature': None, 'tree': {}}, '软粘': {'label:': '好', 'feature': None, 'tree': {}}}}, ' 模糊': {'label:': '坏', 'feature': None, 'tree': {}}, '清晰': {'label:': '好', 'feature': None, 'tree': {}}}}